In [67]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator
plt.style.use('seaborn')
from IPython.display import HTML
%matplotlib inline 
import plotly.express as px
import plotly.graph_objects as go

In [27]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [28]:
confirmed_df.head()


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,50,53,59,70,75,82,114,147,177
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,502,511,581,639,639,701,773,839,825,878
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,150,160,178,178,200,212,226,243,266
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,99,117,129,149,149,197,238,428,566,673


In [29]:
df = pd.DataFrame(confirmed_df)

In [33]:
print(df[['Country/Region','Province/State',]])


Index(['Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20',
       '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20',
       '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20'],
      dtype='object')


In [115]:
confirmed_table = confirmed_df.iloc[0:100,[1,-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]

In [118]:
print(confirmed_table)

            Country/Region  3/17/20
0                 Thailand      177
1                    Japan      878
2                Singapore      266
3                    Nepal        1
4                 Malaysia      673
5                   Canada      103
6                Australia      210
7                Australia       94
8                Australia       78
9                 Cambodia       33
10               Sri Lanka       44
11                 Germany     9257
12                 Finland      321
13    United Arab Emirates       98
14             Philippines      187
15                   India      142
16                   Italy    31506
17                  Sweden     1190
18                   Spain    11748
19               Australia       29
20                 Belgium     1243
21                   Egypt      196
22               Australia        0
23                 Lebanon      120
24                    Iraq      154
25                    Oman       24
26             Afghanistan  

In [89]:
confirmed = confirmed_df.loc[:, cols[4]:cols[-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
total_recovered = [] 

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    mortality_rate.append(death_sum/confirmed_sum)
    total_recovered.append(recovered_sum)

In [63]:
print(confirmed_sum)

197146


In [80]:
fig = go.Figure(data=go.Choropleth(
    locations = confirmed_table['Country/Region'],
    z = confirmed_table['3/17/20'],
    text = confirmed_table['Country/Region'],
    locationmode = 'country names',
    colorscale = 'Reds',
    autocolorscale=True,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
))

In [81]:
fig.show()

In [74]:
latest_confirmed = confirmed_df[dates[-1]]
latest_deaths = deaths_df[dates[-1]]
latest_recoveries = recoveries_df[dates[-1]]
unique_countries =  list(confirmed_df['Country/Region'].unique())
country_confirmed_cases = []
no_cases = []
for i in unique_countries:
    cases = latest_confirmed[confirmed_df['Country/Region']==i].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
        
for i in no_cases:
    unique_countries.remove(i)
    
unique_countries = [k for k, v in sorted(zip(unique_countries, country_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
for i in range(len(unique_countries)):
    country_confirmed_cases[i] = latest_confirmed[confirmed_df['Country/Region']==unique_countries[i]].sum()

In [75]:
# number of cases per country/region
print('Confirmed Cases by Countries/Regions:')
for i in range(len(unique_countries)):
    print(f'{unique_countries[i]}: {country_confirmed_cases[i]} cases')
    

Confirmed Cases by Countries/Regions:
China: 81058 cases
Italy: 31506 cases
Iran: 16169 cases
Spain: 11748 cases
Germany: 9257 cases
Korea, South: 8320 cases
France: 7699 cases
US: 6421 cases
Switzerland: 2700 cases
United Kingdom: 1960 cases
Netherlands: 1708 cases
Norway: 1463 cases
Austria: 1332 cases
Belgium: 1243 cases
Sweden: 1190 cases
Denmark: 1024 cases
Japan: 878 cases
Cruise Ship: 696 cases
Malaysia: 673 cases
Canada: 478 cases
Australia: 452 cases
Portugal: 448 cases
Qatar: 439 cases
Czechia: 396 cases
Greece: 387 cases
Israel: 337 cases
Finland: 321 cases
Brazil: 321 cases
Slovenia: 275 cases
Singapore: 266 cases
Poland: 238 cases
Pakistan: 236 cases
Bahrain: 228 cases
Estonia: 225 cases
Ireland: 223 cases
Iceland: 220 cases
Chile: 201 cases
Egypt: 196 cases
Philippines: 187 cases
Romania: 184 cases
Thailand: 177 cases
Indonesia: 172 cases
Saudi Arabia: 171 cases
Iraq: 154 cases
India: 142 cases
Luxembourg: 140 cases
Kuwait: 130 cases
Lebanon: 120 cases
Peru: 117 cases
Rus